# Prognozowanie sprzedaży Walmartu za pomocą wybranych metod modelowania

Problemem jaki zdecydowaliśmy się opisać i przeanalizować jest przewidywanie sprzedaży sieci dyskontów w USA Walmart, a więc jest to problem regresji. Celem tego projektu jest zbudowanie rozwiązania, które będzie dobrze prognozowało sprzedaż Walmartu, a także wskazywało jakie czynniki mają największe znacznenie. Aby to osiągnąć przetestujemy różne techniki modelowania i wybierzemy najlepszą z nich, a potem zbudujemy na jej podstawie wyniki i prognozę sprzedaży. 

Zbiór danych wejściowych, który posłuży nam do analizy składa się z kilku podzbiorów:
* dane Walmart Sales Forecast pochodzące ze źródła Kaggle (https://www.kaggle.com/datasets/aslanahmedov/walmart-sales-forecast/data) - podstawowy zbiór zawierający informacje na temat sprzedaży, sklepów, a także elementów popytu naturalnego,
* dane Google Trends dotyczące słów powiązanych z badanym problemem,
* dane o wskaźnikach retail trade pochodzące ze strony US Census Bereau (https://www.census.gov/retail/marts/www/timeseries.html?utm_source=chatgpt.com),
* dane na temat świąt państwowych w USA pobrane za pomocą API Nager.Date
* wskaźniki Gross Domestic Product oraz M2 Money Supply pobrane ze strony FRED (https://fred.stlouisfed.org)

Finalnie nasz zbiór danych surowych został przygotowany za pomocą skryptu **data_downloading.ipynb** i składa się z następujących plików:
* train_data_final_df.csv - główny zbiór danych, który posłuży do trenowania i walidacji modeli,
* forecasts_data_final_df.csv - zbiór danych, który posłuży do testowania modeli i generowania prognoz.

Oba zbiory składają się z tej samej ilości takich samych zmiennych, natomiast różnią się jedynie zakresem dat. Zbiór do trenowania zaczyna się od 2010-02-07 i kończy na 2012-10-28, a zbiór do prognoz zaczyna się od 2012-11-04 i kończy na 2013-07-28. Warto również dodać, że oprócz daty mamy również informację o numerze sklepu, więc nasz problem dotyczy regresji, a dokładniej regresji panelowej, gdyż mamy dane rozłożone w czasie dla różnych jednostek. Poniżej znajduje się opis każdej zmiennej surowej, który pomoże w lepszym zrozumieniu zbioru oraz wyjaśnieniu niektórych wskaźników.

Opis zmiennych surowych ma na celu przybliżenie ich znaczenia oraz kontekstu, w jakim będą używane. Po wstępnej obróbce danych i przygotowywaniu ich do analizy, zmienne te mogą ulec pewnym zmianom.

1. Store - zmienna oznaczająca numer sklepu. Jest to ID, które pozwoli rozróżniać, którego sklepu dotyczą dane, a także posłuży jako element indeksu przy modelowaniu.
2. Dept - zmienna oznaczająca numer działu, który wygenerował konkretną sprzedaż. Można oczekiwać, że zmienna ta wykaże, że na niektórych działach sprzedaż jest większa (np. Elektronika vs Ubrania).
3. Date - zmienna oznaczająca datę w granulacji W-SUN czyli tygodniowa z niedzielą jako początkiem tygodnia. Dla każdego sklepu zakres dat powinien być taki sam, co zostanie sprawdzone w procesie obróbki.
4. IsHoliday - zmienna mówiąca o tym czy w danym tygodniu było jakieś święto.
5. Temperature - temperatura w danym tygodniu.
6. Fuel_Price - cena paliwa.
7. MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown5 - są to zmienne oznaczające konkretne typy promocji. Mogą być powiązane z jakimiś świętami lub być po prostu ogólną promocją. Im większe tym większa była promocja.
8. CPI - wskaźnik inflacji.
9. Unemployment - stopa bezrobocia.
10. Type - typ sklepu. Jest on prawdopodobnie skorelowany z rozmiarem, ponieważ typ A to sklepy w centrach handlowych i dużych skupiskach, typ B to średnie dyskonty, a typ C to mniejsze sklepy sąsiedzkie.
11. Size - rozmiar sklepu.
12. valentines_day_gt - zmienna dotycząca Google Trends dla słowa valentine day w USA. Przyjmuje ona wartości od 0 do 100. Google buduje te zmienne w ten sposób, że na wybranym przez nas okresie znajduje max(wyświetleń słowa) i wszystkie wartości z naszego szeregu dzieli na tego maksa i mnoży przez 100. Przez co dostajemy bardziej wskaźnik natężenia ruchu w internecie dla danego słowa, a nie jego wyświetlenia.
13. sams_club_competition_gt, costco_competition_gt, dollar_tree_competition_gt, kroger_competition_gt, target_shop_competition_gt - zmienne GT dla słówa Sam's club, Costco, Dollar Tree, Kroger oraz Target Shop. Są to najwięksi konkurenci Walmarta w USA.
14. black_friday_gt, cyber_monday_gt, back_to_school_gt - zmienne GT związane z jakimiś promocyjnymi okresami jak Black Friday czy powrót do szkoły.
15. supermarket_near_me_gt, supermarket_gt - zmienne GT sklep obok mnie oraz supermarket.
16. Retail_Trade_Not_Seasonally_Adjusted, Retail_Trade_Seasonally_Adjusted - wskaźnik retail trade, który pokazuje konkdycję dyskontów, może pokazywać boom gospodarczy lub recesję z wyprzedzeniem. Sezonowy uwzględnia sezonowość, a ten surowy pokazuje czyste przychody wszystkich dyskontów.
17. m2_money_supply_indicator - wskaźnik podaży pieniądza M2.
18. GDP - Gross Domestic Product.
19. HolidayName - nazwa święta jako string lub NA gdy brak święta. Zmienna ma na celu bardziej szczegółowe sprawdzenie jak konkretne święta wpływają na sprzedaż.